# Importar Librerías 


In [64]:
# Manipulación de datos 
import pandas as pd 
import numpy as np 
import math 
import datetime as dt 
# Visualización de datos 
import seaborn as sns 
import matplotlib.pyplot as plt 

# Importar Datasets 
Los datasets contienen información sobre el uso del servicio de telefonía virtual CallMeMaybe. Sus clientes son organizaciones que necesitan distribuir gran cantidad de llamadas entrantes entre varios operadores, o realizar llamadas salientes a través de sus operadores. Los operadores también pueden realizar llamadas internas para comunicarse entre ellos. Estas llamadas se realizan a través de la red de CallMeMaybe.

## Telecom Dataset 
- `user_id`: ID de la cuenta de cliente
- `date`: fecha en la que se recuperaron las estadísticas
- `direction`: "dirección" de llamada (`out` para saliente, `in` para entrante)
- `internal`: si la llamada fue interna (entre los operadores de un cliente o clienta)
- `operator_id`: identificador del operador
- `is_missed_call`: si fue una llamada perdida
- `calls_count`: número de llamadas
- `call_duration`: duración de la llamada (sin incluir el tiempo de espera)
- `total_call_duration`: duración de la llamada (incluido el tiempo de espera)

In [65]:
data = pd.read_csv('telecom_dataset_us.csv')
data.head(10)

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
0,166377,2019-08-04 00:00:00+03:00,in,False,NaN,True,2,0,4
1,166377,2019-08-05 00:00:00+03:00,out,True,880022.0,True,3,0,5
2,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,True,1,0,1
3,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,False,1,10,18
4,166377,2019-08-05 00:00:00+03:00,out,False,880022.0,True,3,0,25
5,166377,2019-08-05 00:00:00+03:00,out,False,880020.0,False,2,3,29
6,166377,2019-08-05 00:00:00+03:00,out,False,880020.0,True,8,0,50
7,166377,2019-08-05 00:00:00+03:00,in,False,NaN,True,6,0,35
8,166377,2019-08-05 00:00:00+03:00,out,False,880020.0,True,8,0,50
9,166377,2019-08-06 00:00:00+03:00,in,False,NaN,True,4,0,62


## Telecom Clients 
- `user_id`: ID de usuario/a
- `tariff_plan`: tarifa actual de la clientela
- `date_start`: fecha de registro de la clientela

In [66]:
clients = pd.read_csv('telecom_clients_us.csv')
clients.head(10)

,user_id,tariff_plan,date_start
0,166713,A,2019-08-15
1,166901,A,2019-08-23
2,168527,A,2019-10-29
3,167097,A,2019-09-01
4,168193,A,2019-10-16
5,167764,A,2019-09-30
6,167050,A,2019-08-29
7,168252,A,2019-10-17
8,168495,A,2019-10-28
9,167879,A,2019-10-03


# Preprocesamiento de Datos 

## Valores Ausentes 

### Data_Df

In [67]:
# Usar 'isna()' para ver la cantidad de valores ausentes en las distintas columnas del df
data.isna().sum()

user_id                   0
date                      0
direction                 0
internal                117
operator_id            8172
is_missed_call            0
calls_count               0
call_duration             0
total_call_duration       0
dtype: int64

Vemos que la columna con el identificador de los operarios tiene varios valores ausentes, por lo que al ser esta la columna con los valores ausentes lo mejor es eliminar todos estos valores ausentes ya que estos no nos ayudan con nuestro objetivo que es identificar a los operarios ineficientes y en este caso hace falta una de las cosas más importantes que es el identificador del usuario

In [68]:
# Eliminar valores duplicados 
data.dropna(subset='operator_id', inplace=True)

### Clients_Df

In [69]:
clients.isna().sum()

user_id        0
tariff_plan    0
date_start     0
dtype: int64

El dataset con la información de los clientes no tiene valores ausentes

## Tipos de datos correctos

### Data_DF

In [70]:
# Ver información general del dataset 
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45730 entries, 1 to 53900
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              45730 non-null  int64  
 1   date                 45730 non-null  object 
 2   direction            45730 non-null  object 
 3   internal             45670 non-null  object 
 4   operator_id          45730 non-null  float64
 5   is_missed_call       45730 non-null  bool   
 6   calls_count          45730 non-null  int64  
 7   call_duration        45730 non-null  int64  
 8   total_call_duration  45730 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 3.2+ MB


En la información general del df se logra observar que hay varias columnas con el tipo de dato incorrecto para lo que significan, por ejemplo: las columnas que guardan los identificadores tanto de los usuarios como de los operadores son de tipo 'int' y para este tipo de información es mejor que se trabajen como strings, también con la columna date, esta debería ser de tipo datetime.

In [71]:
# Convertir los ID a tipo object
data[['user_id','operator_id']] = data[['user_id','operator_id']].astype(str)
# Convertir la columna referente a las fechas a tipo datetime  
data['date'] = pd.to_datetime(data['date'])
# Eliminar la zona horaria
data['date'] = data['date'].dt.tz_convert(None)
# Reemplazar la columna existente con solo la fecha en tipo datetime
data['date'] = pd.to_datetime(data['date'].dt.date)

In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45730 entries, 1 to 53900
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id              45730 non-null  object        
 1   date                 45730 non-null  datetime64[ns]
 2   direction            45730 non-null  object        
 3   internal             45670 non-null  object        
 4   operator_id          45730 non-null  object        
 5   is_missed_call       45730 non-null  bool          
 6   calls_count          45730 non-null  int64         
 7   call_duration        45730 non-null  int64         
 8   total_call_duration  45730 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int64(3), object(4)
memory usage: 3.2+ MB


### Clients_Df

In [73]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      732 non-null    int64 
 1   tariff_plan  732 non-null    object
 2   date_start   732 non-null    object
dtypes: int64(1), object(2)
memory usage: 17.3+ KB


En el df de los clientes solo se tiene que cambiar el tipo de datos para la columna 'user_id' y 'date_start' a tipo object y datetime respectivamente

In [74]:
# Convertir a tipo object
clients['user_id'] = clients['user_id'].astype(str)
# Convertir a datetime
clients['date_start'] = pd.to_datetime(clients['date_start'])

## Valores Ausentes 

### Data_Df

In [75]:
data.duplicated().sum()

4184

In [76]:
data = data.drop_duplicates()

### Clients_Df

In [78]:
clients.duplicated().sum()

0

# Análisis Exploratorio de Datos 


## Estadísticas descriptivas del dataset